In [8]:
# БЛОК 1: ИМПОРТ БИБЛИОТЕК
import tkinter as tk
from tkinter import ttk, messagebox, filedialog, simpledialog
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
import json
from datetime import datetime
import os
import random
from collections import defaultdict
import sqlite3
from contextlib import contextmanager

# БЛОК 2: КЛАСС ПРОДУКТА ДЛЯ ПРЕДСТАВЛЕНИЯ ТОВАРА
class Product:
    """
    Класс для представления товара в системе рекомендаций
    """
    def __init__(self, id, name, category, price, description, image="📦", tags=None):
        self.id = id
        self.name = name
        self.category = category
        self.price = price
        self.description = description
        self.image = image
        self.tags = tags or []

# БЛОК 3: ОСНОВНОЙ КЛАСС СИСТЕМЫ РЕКОМЕНДАЦИЙ
class EnhancedRecommendationSystem:
    """
    Система рекомендаций с использованием нескольких алгоритмов:
    - Коллаборативная фильтрация
    - Контентные рекомендации
    - Кластеризация
    - Случайный лес
    - Гибридный подход
    """
    def __init__(self):
        # Инициализация каталога товаров и данных пользователя
        self.products = self._generate_products()
        self.user_interactions = {
            'views': [],      # Просмотренные товары
            'favorites': [],  # Избранные товары
            'cart': [],       # Товары в корзине
            'purchases': [],  # Купленные товары
            'ratings': {}     # Оценки товаров (product_id: rating)
        }
        # Веса взаимодействий для расчета предпочтений пользователя
        self.interaction_weights = {
            'view': 1,       # Просмотр товара
            'favorite': 3,   # Добавление в избранное
            'cart': 5,       # Добавление в корзину
            'purchase': 8,   # Покупка товара
            'rating': 10     # Оценка товара
        }
        # Генерация исторических данных для обучения моделей
        self.historical_data = self._generate_enhanced_historical_data()
        # Расчет признаков товаров для контентных рекомендаций
        self.tfidf_matrix, self.svd_features = self._calculate_enhanced_content_features()
        # Матрица схожести пользователей для коллаборативной фильтрации
        self.user_similarity_matrix = self._calculate_user_similarity_matrix()
        # История метрик для анализа производительности
        self.metrics_history = []
        # Текущие метрики различных алгоритмов
        self.metrics = {
            'cold_start': {},       # Метрики для холодного старта
            'personalized': {},     # Персонализированные рекомендации
            'content_based': {},    # Контентные рекомендации
            'hybrid': {},           # Гибридные рекомендации
            'cluster_based': {},    # Кластерные рекомендации
            'random_forest': {}     # Рекомендации на основе случайного леса
        }
        self.current_user_id = 1
        self.feedback_data = []
        # Инициализация базы данных
        self._init_database()
        # Обучение моделей машинного обучения
        self.cluster_model = self._train_cluster_model()
        self.rf_model = self._train_random_forest_model()
        
        # Объяснения метрик для пользовательского интерфейса
        self.metric_explanations = {
            'precision@10': 'Доля релевантных товаров в топ-10 рекомендациях',
            'precision': 'Точность рекомендаций (релевантные/все рекомендованные)',
            'recall': 'Полнота (сколько релевантных товаров нашли)',
            'ndcg@10': 'Качество ранжирования (учитывает порядок рекомендаций)',
            'coverage': 'Разнообразие рекомендованных товаров от общего каталога',
            'diversity': 'Разнообразие категорий в рекомендациях',
            'serendipity': 'Неожиданные, но релевантные рекомендации',
            'personalization_score': 'Степень персонализации под пользователя',
            'cluster_coherence': 'Схожесть товаров в рекомендациях',
            'model_confidence': 'Уверенность модели в рекомендациях',
            'interactions_count': 'Количество взаимодействий пользователя',
            'total_recommendations': 'Общее количество рекомендаций'
        }
        
    # БЛОК 4: МЕТОДЫ РАБОТЫ С БАЗОЙ ДАННЫХ
    def _init_database(self):
        """Инициализация базы данных SQLite для хранения взаимодействий пользователей"""
        with self._get_db_connection() as conn:
            # Таблица для хранения всех пользовательских взаимодействий
            conn.execute('''
                CREATE TABLE IF NOT EXISTS user_interactions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER,
                    product_id INTEGER,
                    interaction_type TEXT,
                    rating INTEGER,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            # Таблица для хранения обратной связи по рекомендациям
            conn.execute('''
                CREATE TABLE IF NOT EXISTS user_feedback (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER,
                    recommendation_type TEXT,
                    product_id INTEGER,
                    feedback_score INTEGER,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            conn.commit()
    
    @contextmanager
    def _get_db_connection(self):
        """Контекстный менеджер для подключения к базе данных"""
        conn = sqlite3.connect('recommendation_system.db')
        try:
            yield conn
        finally:
            conn.close()
    
    def _save_interaction(self, interaction_type, product_id, rating=None):
        """Сохранение взаимодействия пользователя в базу данных"""
        with self._get_db_connection() as conn:
            conn.execute(
                'INSERT INTO user_interactions (user_id, product_id, interaction_type, rating) VALUES (?, ?, ?, ?)',
                (self.current_user_id, product_id, interaction_type, rating)
            )
            conn.commit()
    
    def _save_feedback(self, recommendation_type, product_id, feedback_score):
        """Сохранение обратной связи пользователя в базу данных"""
        with self._get_db_connection() as conn:
            conn.execute(
                'INSERT INTO user_feedback (user_id, recommendation_type, product_id, feedback_score) VALUES (?, ?, ?, ?)',
                (self.current_user_id, recommendation_type, product_id, feedback_score)
            )
            conn.commit()
    
    # БЛОК 5: ГЕНЕРАЦИЯ ДАННЫХ ДЛЯ СИСТЕМЫ
    def _generate_products(self):
        """Генерация каталога товаров с тегами и категориями"""
        products = []
        categories = {
            'Электроника': [
                ('Смартфон Galaxy S23', 59999, 'Флагманский смартфон с камерой 200MP', '📱', ['смартфон', 'флагман', 'камера', 'android', 'самсунг']),
                ('Ноутбук Dell XPS', 89999, 'Ультрабук для работы и развлечений', '💻', ['ноутбук', 'ультрабук', 'работа', 'windows', 'dell']),
                ('Наушники Sony WH-1000XM5', 29999, 'Беспроводные с шумоподавлением', '🎧', ['наушники', 'беспроводные', 'шумоподавление', 'аудио', 'sony']),
                ('Планшет iPad Air', 54999, 'Планшет с дисплеем Liquid Retina', '📱', ['планшет', 'apple', 'retina', 'мобильный', 'айпад']),
                ('Умные часы Apple Watch', 39999, 'Фитнес-трекер и уведомления', '⌚', ['часы', 'фитнес', 'смартфон', 'apple', 'трекер']),
                ('Фотокамера Canon EOS R6', 129999, 'Зеркальная камера для профессионалов', '📷', ['камера', 'зеркальная', 'профессиональная', 'фото', 'canon']),
                ('Игровая консоль PS5', 49999, 'Игровая приставка нового поколения', '🎮', ['консоль', 'игры', 'развлечения', 'sony', 'playstation']),
                ('Монитор Samsung 32"', 34999, '4K монитор для работы и игр', '🖥️', ['монитор', '4k', 'игры', 'работа', 'самсунг']),
                ('Клавиатура механическая', 7999, 'Игровая клавиатура с RGB подсветкой', '⌨️', ['клавиатура', 'механическая', 'игры', 'rgb', 'гейминг']),
                ('Мышь беспроводная', 3999, 'Эргономичная мышь для продуктивной работы', '🐁', ['мышь', 'беспроводная', 'эргономика', 'работа', 'офис']),
                ('Портативная колонка JBL', 8999, 'Мощная Bluetooth колонка с бассейном', '🔊', ['колонка', 'bluetooth', 'портативная', 'музыка', 'jbl']),
                ('Электронная книга Kindle', 12999, 'Читалка с подсветкой и защитой от воды', '📖', ['книга', 'электронная', 'читалка', 'amazon', 'kindle']),
                ('Внешний аккумулятор 20000mAh', 4999, 'Мощный пауэрбанк для зарядки устройств', '🔋', ['аккумулятор', 'пауэрбанк', 'зарядка', 'мобильный']),
                ('Роутер Wi-Fi 6', 6999, 'Высокоскоростной маршрутизатор нового поколения', '📡', ['роутер', 'wi-fi', 'интернет', 'сеть', 'маршрутизатор']),
                ('Дрон DJI Mini 3', 89999, 'Компактный дрон с камерой 4K', '🚁', ['дрон', 'камера', 'квадрокоптер', 'dji', 'аэросъемка']),
            ],
            'Одежда': [
                ('Куртка зимняя', 12999, 'Теплая куртка с мембраной', '🧥', ['куртка', 'зимняя', 'тепло', 'мембрана', 'одежда']),
                ('Джинсы Levi\'s', 5999, 'Классические джинсы из денима', '👖', ['джинсы', 'классика', 'levis', 'повседневная', 'деним']),
                ('Кроссовки Nike', 8999, 'Спортивные кроссовки для бега', '👟', ['кроссовки', 'спорт', 'бег', 'nike', 'обувь']),
                ('Свитер шерстяной', 4999, 'Уютный свитер из натуральной шерсти', '🧶', ['свитер', 'шерсть', 'уют', 'зима', 'теплый']),
                ('Рубашка офисная', 3999, 'Классическая белая рубашка', '👔', ['рубашка', 'офис', 'классика', 'белая', 'деловая']),
                ('Пальто осеннее', 15999, 'Стильное пальто для прохладной погоды', '🧥', ['пальто', 'осень', 'стиль', 'тепло', 'верхняя']),
                ('Футболка хлопковая', 1999, 'Удобная футболка из натурального хлопка', '👕', ['футболка', 'хлопок', 'повседневная', 'комфорт', 'базовая']),
                ('Шорты спортивные', 2999, 'Легкие шорты для тренировок', '🩳', ['шорты', 'спорт', 'тренировки', 'легкие', 'лето']),
                ('Платье вечернее', 8999, 'Элегантное платье для особых случаев', '👗', ['платье', 'вечернее', 'элегантное', 'особый случай', 'выход']),
                ('Костюм спортивный', 6999, 'Комфортный костюм для занятий спортом', '👕', ['костюм', 'спортивный', 'комфорт', 'тренировки', 'активный']),
                ('Пуховик зимний', 18999, 'Теплый пуховик для морозной погоды', '🧥', ['пуховик', 'зима', 'мороз', 'теплый', 'пух']),
                ('Юбка карандаш', 4599, 'Классическая юбка для офиса', '👗', ['юбка', 'офис', 'классика', 'деловая', 'карандаш']),
                ('Бомбер кожаный', 12999, 'Стильная кожаная куртка-бомбер', '🧥', ['бомбер', 'кожа', 'стиль', 'куртка', 'мужская']),
                ('Кардиган вязаный', 5599, 'Уютный кардиган на пуговицах', '🧶', ['кардиган', 'вязаный', 'уют', 'осень', 'свитер']),
                ('Брюки классические', 6999, 'Деловые брюки для офиса', '👖', ['брюки', 'офис', 'классика', 'деловые', 'формальные']),
            ],
            'Книги': [
                ('Python для анализа данных', 2499, 'Учебник по Data Science на Python', '📚', ['python', 'анализ данных', 'data science', 'программирование', 'учебник']),
                ('Чистый код', 1999, 'Создание, анализ и рефакторинг', '📖', ['программирование', 'код', 'рефакторинг', 'разработка', 'software']),
                ('Алгоритмы', 2999, 'Построение и анализ алгоритмов', '📕', ['алгоритмы', 'программирование', 'компьютерные науки', 'математика', 'структуры']),
                ('Машинное обучение', 3499, 'Введение в ML и нейросети', '📗', ['машинное обучение', 'ml', 'нейросети', 'ai', 'искусственный интеллект']),
                ('Дизайн паттерны', 2199, 'Паттерны проектирования ПО', '📘', ['паттерны', 'дизайн', 'программирование', 'разработка', 'архитектура']),
                ('Гарри Поттер', 1899, 'Фэнтези роман о юном волшебнике', '📙', ['фэнтези', 'роман', 'гарри поттер', 'магия', 'приключения']),
                ('Война и мир', 2299, 'Классический роман Льва Толстого', '📔', ['классика', 'роман', 'толстой', 'литература', 'русская']),
                ('Преступление и наказание', 1999, 'Роман Федора Достоевского', '📓', ['классика', 'роман', 'достоевский', 'литература', 'психология']),
                ('1984', 1799, 'Антиутопия Джорджа Оруэлла', '📒', ['антиутопия', 'роман', 'оруэлл', 'классика', 'политика']),
                ('Мастер и Маргарита', 2099, 'Роман Михаила Булгакова', '📗', ['классика', 'роман', 'булгаков', 'магия', 'мистика']),
                ('Сто лет одиночества', 2199, 'Роман Габриэля Гарсиа Маркеса', '📘', ['магический реализм', 'роман', 'маркес', 'латиноамериканская']),
                ('Три товарища', 1899, 'Роман Эриха Марии Ремарка', '📙', ['роман', 'ремарк', 'дружба', 'любовь', 'классика']),
                ('Маленький принц', 999, 'Философская сказка Антуана де Сент-Экзюпери', '📕', ['сказка', 'философия', 'детская', 'классика', 'французская']),
                ('Шерлок Холмс', 2399, 'Собрание рассказов о знаменитом детективе', '📔', ['детектив', 'холмс', 'конан дойл', 'классика', 'расследование']),
                ('Властелин колец', 3999, 'Эпическая фэнтези сага Дж. Р. Р. Толкина', '📘', ['фэнтези', 'толкин', 'эпическая', 'приключения', 'кольцо']),
            ],
            'Дом и сад': [
                ('Кофемашина Delonghi', 34999, 'Автоматическая кофемашина', '☕', ['кофемашина', 'кофе', 'автоматическая', 'кухня', 'delonghi']),
                ('Пылесос Dyson', 45999, 'Беспроводной пылесос', '🧹', ['пылесос', 'беспроводной', 'уборка', 'dyson', 'техника']),
                ('Мультиварка', 7999, 'Программируемая мультиварка 5л', '🍲', ['мультиварка', 'кухня', 'готовка', 'программируемая', 'еда']),
                ('Набор посуды', 9999, 'Набор из 12 предметов', '🍽️', ['посуда', 'кухня', 'набор', 'готовка', 'кастрюли']),
                ('Увлажнитель воздуха', 5999, 'Ультразвуковой увлажнитель', '💧', ['увлажнитель', 'воздух', 'здоровье', 'комфорт', 'климат']),
                ('Диван угловой', 45999, 'Удобный угловой диван для гостиной', '🛋️', ['диван', 'угловой', 'гостиная', 'комфорт', 'мебель']),
                ('Светильник LED', 2999, 'Современный светильник с LED подсветкой', '💡', ['светильник', 'led', 'освещение', 'интерьер', 'лампа']),
                ('Шторы blackout', 4999, 'Затемняющие шторы для спальни', '🪟', ['шторы', 'blackout', 'спальня', 'темнота', 'окно']),
                ('Ковер персидский', 12999, 'Красивый ковер ручной работы', '🧶', ['ковер', 'персидский', 'интерьер', 'ручная работа', 'декор']),
                ('Горшки для цветов', 1999, 'Набор керамических горшков для растений', '🪴', ['горшки', 'цветы', 'сад', 'керамика', 'растения']),
                ('Электрический гриль', 8999, 'Компактный гриль для дома', '🍖', ['гриль', 'электрический', 'готовка', 'мясо', 'кухня']),
                ('Набор инструментов', 6999, 'Профессиональный набор для ремонта', '🛠️', ['инструменты', 'ремонт', 'набор', 'профессиональный', 'дом']),
                ('Садовые качели', 15999, 'Уличные качели для дачи', '🌳', ['качели', 'сад', 'уличные', 'отдых', 'дача']),
                ('Обеденный стол', 23999, 'Деревянный стол на 6 персон', '🍽️', ['стол', 'обеденный', 'деревянный', 'кухня', 'мебель']),
                ('Стиральная машина', 34999, 'Автоматическая стиральная машина', '🧼', ['стиральная', 'машина', 'белье', 'автоматическая', 'техника']),
            ],
            'Спорт': [
                ('Гантели 10кг', 3999, 'Разборные гантели для дома', '🏋️', ['гантели', 'спорт', 'тренировки', 'дом', 'сила']),
                ('Коврик для йоги', 1999, 'Нескользящий коврик 183x61см', '🧘', ['коврик', 'йога', 'фитнес', 'нескользящий', 'тренировки']),
                ('Велосипед горный', 29999, 'MTB велосипед 21 скорость', '🚴', ['велосипед', 'горный', 'спорт', 'mtb', 'кардио']),
                ('Скакалка', 799, 'Скоростная скакалка для фитнеса', '🪢', ['скакалка', 'фитнес', 'кардио', 'тренировки', 'скоростная']),
                ('Гиря 16кг', 2999, 'Чугунная гиря для кроссфита', '⚫', ['гиря', 'кроссфит', 'тренировки', 'сила', 'чугун']),
                ('Беговая дорожка', 59999, 'Электрическая беговая дорожка', '🏃', ['беговая дорожка', 'кардио', 'тренировки', 'дом', 'бег']),
                ('Эспандер резиновый', 999, 'Набор резиновых эспандеров', '💪', ['эспандер', 'резиновый', 'тренировки', 'сила', 'набор']),
                ('Ролик для пресса', 1499, 'Массажный ролик для тренировок', '🎽', ['ролик', 'пресс', 'массаж', 'тренировки', 'массажный']),
                ('Спортивная бутылка', 799, 'Бутылка для воды с фильтром', '💧', ['бутылка', 'спорт', 'вода', 'фильтр', 'гидратация']),
                ('Фитнес-браслет', 4999, 'Трекер активности и сна', '⌚', ['браслет', 'фитнес', 'трекер', 'здоровье', 'активность']),
                ('Теннисная ракетка', 5999, 'Профессиональная ракетка для большого тенниса', '🎾', ['ракетка', 'теннис', 'спорт', 'профессиональная', 'игра']),
                ('Баскетбольный мяч', 2999, 'Мяч для профессиональных игр', '🏀', ['мяч', 'баскетбол', 'спорт', 'профессиональный', 'игра']),
                ('Боксерские перчатки', 3999, 'Перчатки для тренировок по боксу', '🥊', ['перчатки', 'бокс', 'тренировки', 'спорт', 'единоборства']),
                ('Плавательные очки', 1499, 'Герметичные очки для плавания', '🥽', ['очки', 'плавание', 'бассейн', 'герметичные', 'спорт']),
                ('Рюкзак для треккинга', 7999, 'Вместительный рюкзак для походов', '🎒', ['рюкзак', 'треккинг', 'поход', 'вместительный', 'спорт']),
            ],
            'Красота и здоровье': [
                ('Электрическая зубная щетка', 4999, 'Щетка с технологией sonic', '🦷', ['зубная щетка', 'электрическая', 'гигиена', 'здоровье', 'sonic']),
                ('Фен профессиональный', 6999, 'Мощный фен для укладки волос', '💇', ['фен', 'профессиональный', 'волосы', 'укладка', 'красота']),
                ('Набор косметики', 8999, 'Люкс набор косметических средств', '💄', ['косметика', 'набор', 'люкс', 'красота', 'уход']),
                ('Массажер для шеи', 3999, 'Электронный массажер с подогревом', '💆', ['массажер', 'шея', 'расслабление', 'здоровье', 'электронный']),
                ('Весы умные', 2999, 'Весы с анализом состава тела', '⚖️', ['весы', 'умные', 'здоровье', 'анализ', 'фитнес']),
                ('Эпилятор', 5999, 'Беспроводной эпилятор для дома', '✨', ['эпилятор', 'беспроводной', 'красота', 'уход', 'депиляция']),
                ('Зеркало с подсветкой', 4599, 'Косметическое зеркало с LED', '👩', ['зеркало', 'косметическое', 'подсветка', 'красота', 'уход']),
                ('Набор для маникюра', 2999, 'Профессиональный набор для ногтей', '💅', ['маникюр', 'набор', 'профессиональный', 'ногти', 'уход']),
                ('Аромадиффузор', 2499, 'Ультразвуковой диффузор для масел', '🌿', ['диффузор', 'ароматерапия', 'масла', 'релаксация', 'здоровье']),
                ('Щетка для лица', 1999, 'Электронная щетка для очистки кожи', '🧼', ['щетка', 'лицо', 'очистка', 'кожа', 'уход']),
            ],
            'Игрушки и хобби': [
                ('Конструктор LEGO', 3999, 'Большой набор для сборки', '🧱', ['конструктор', 'lego', 'игрушка', 'творчество', 'сборка']),
                ('Настольная игра', 2499, 'Стратегическая игра для компании', '🎲', ['настольная игра', 'стратегия', 'компания', 'развлечения', 'досуг']),
                ('Пазл 1000 деталей', 1599, 'Большой пазл с красивым изображением', '🧩', ['пазл', 'головоломка', 'досуг', 'творчество', 'детали']),
                ('Радиоуправляемый вертолет', 5999, 'Вертолет с гироскопом', '🚁', ['вертолет', 'радиоуправляемый', 'игрушка', 'гироскоп', 'полет']),
                ('Набор для вышивания', 1299, 'Набор для творчества и рукоделия', '🧵', ['вышивание', 'набор', 'творчество', 'рукоделие', 'досуг']),
                ('Модель для сборки', 2999, 'Детализированная модель автомобиля', '🚗', ['модель', 'сборка', 'автомобиль', 'хобби', 'детали']),
                ('Дротики для дартса', 999, 'Профессиональный набор дротиков', '🎯', ['дартс', 'дротики', 'игра', 'развлечения', 'спорт']),
                ('3D-ручка', 3999, 'Ручка для создания 3D-объектов', '✏️', ['3d-ручка', 'творчество', 'рисование', 'технологии', 'хобби']),
                ('Набор фокусника', 1999, 'Набор для обучения фокусам', '🎩', ['фокусы', 'набор', 'обучение', 'развлечения', 'магия']),
                ('Железная дорога', 7999, 'Электрическая железная дорога', '🚂', ['железная дорога', 'игрушка', 'поезд', 'электрическая', 'детская']),
            ],
            'Автотовары': [
                ('Автомобильный холодильник', 7999, 'Компактный холодильник для авто', '🚗', ['холодильник', 'автомобильный', 'поездки', 'еда', 'компактный']),
                ('Видеорегистратор', 5999, 'Регистратор с ночной съемкой', '📹', ['видеорегистратор', 'авто', 'безопасность', 'съемка', 'ночная']),
                ('Компрессор автомобильный', 2999, 'Портативный компрессор для шин', '💨', ['компрессор', 'автомобильный', 'шины', 'насос', 'портативный']),
                ('Чехлы для сидений', 3999, 'Кожаные чехлы для автомобиля', '🚘', ['чехлы', 'авто', 'сиденья', 'кожа', 'защита']),
                ('Аккумулятор пусковой', 8999, 'Пусковое устройство для авто', '🔋', ['аккумулятор', 'пусковой', 'авто', 'запуск', 'устройство']),
                ('Навигатор', 6999, 'GPS навигатор с обновлениями', '🧭', ['навигатор', 'gps', 'авто', 'карты', 'навигация']),
                ('Автошампунь', 799, 'Профессиональное средство для мойки', '🧽', ['шампунь', 'авто', 'мойка', 'очистка', 'средство']),
                ('Коврики в салон', 1999, 'Резиновые коврики для автомобиля', '🚗', ['коврики', 'авто', 'салон', 'резиновые', 'защита']),
                ('Зарядное устройство', 2499, 'Устройство для зарядки в авто', '🔌', ['зарядное', 'авто', 'устройство', 'usb', 'электроника']),
                ('Домкрат гидравлический', 3999, 'Профессиональный домкрат 2т', '🔧', ['домкрат', 'гидравлический', 'авто', 'ремонт', 'профессиональный']),
            ]
        }
        
        product_id = 1
        for category, items in categories.items():
            for name, price, desc, emoji, tags in items:
                products.append(Product(product_id, name, category, price, desc, emoji, tags))
                product_id += 1
        
        return products
    
    def _generate_enhanced_historical_data(self):
        """Генерация исторических данных с паттернами поведения пользователей"""
        n_users = 300
        n_products = len(self.products)
        
        # Создаем паттерны предпочтений пользователей
        user_patterns = {}
        categories = list(set(p.category for p in self.products))
        
        for user_id in range(1, n_users + 1):
            preferred_categories = random.sample(categories, random.randint(1, 3))
            user_patterns[user_id] = {
                'preferred_categories': preferred_categories,
                'price_preference': random.choice(['budget', 'medium', 'premium']),
                'interaction_intensity': random.uniform(0.4, 0.95),
                'preferred_tags': random.sample(['техника', 'спорт', 'мода', 'книги', 'дом', 'красота', 'игры', 'авто'], 2)
            }
        
        data = []
        for user_id in range(1, n_users + 1):
            pattern = user_patterns[user_id]
            
            n_interactions = int(random.randint(20, 60) * pattern['interaction_intensity'])
            
            # Товары, соответствующие предпочтениям пользователя
            preferred_products = [p for p in self.products 
                                if p.category in pattern['preferred_categories'] 
                                or any(tag in p.tags for tag in pattern['preferred_tags'])]
            
            other_products = [p for p in self.products if p not in preferred_products]
            
            n_preferred = int(n_interactions * 0.8)
            n_other = n_interactions - n_preferred
            
            preferred_ids = [p.id for p in random.sample(preferred_products, min(n_preferred, len(preferred_products)))]
            other_ids = [p.id for p in random.sample(other_products, min(n_other, len(other_products)))]
            
            product_ids = preferred_ids + other_ids
            
            for prod_id in product_ids:
                product = next(p for p in self.products if p.id == prod_id)
                
                interaction_types = ['view']
                
                # Вероятности взаимодействий в зависимости от соответствия предпочтениям
                is_preferred = (product.category in pattern['preferred_categories'] or 
                              any(tag in product.tags for tag in pattern['preferred_tags']))
                
                fav_prob = 0.8 if is_preferred else 0.2
                if random.random() < fav_prob:
                    interaction_types.append('favorite')
                
                cart_prob = 0.6 if is_preferred else 0.15
                if random.random() < cart_prob:
                    interaction_types.append('cart')
                
                purchase_prob = 0.4 if is_preferred else 0.1
                if random.random() < purchase_prob:
                    interaction_types.append('purchase')
                
                for interaction in interaction_types:
                    data.append({
                        'user_id': user_id,
                        'product_id': prod_id,
                        'interaction_type': interaction,
                        'timestamp': datetime.now().isoformat()
                    })
        
        df = pd.DataFrame(data)
        df.to_csv('enhanced_historical_interactions.csv', index=False)
        return df
    
    # БЛОК 6: МЕТОДЫ РАСЧЕТА ПРИЗНАКОВ И МОДЕЛЕЙ МАШИННОГО ОБУЧЕНИЯ
    def _calculate_enhanced_content_features(self):
        """Расчет контентных признаков с учетом тегов и категорий"""
        product_data = []
        for p in self.products:
            extended_description = f"{p.category} {p.name} {p.description} {' '.join(p.tags)} "
            
            if p.category == 'Электроника':
                extended_description += "техника гаджет технология современный цифровой электроника "
            elif p.category == 'Одежда':
                extended_description += "мода стиль одежда комфорт качественный ткань "
            elif p.category == 'Книги':
                extended_description += "литература чтение образование знания автор книга "
            elif p.category == 'Дом и сад':
                extended_description += "интерьер уют дом комфорт качество быт "
            elif p.category == 'Спорт':
                extended_description += "тренировка здоровье активность фитнес спорт "
            elif p.category == 'Красота и здоровье':
                extended_description += "красота здоровье уход косметика гигиена "
            elif p.category == 'Игрушки и хобби':
                extended_description += "игрушки хобби развлечения творчество досуг "
            elif p.category == 'Автотовары':
                extended_description += "авто автомобиль машина транспорт запчасти "
            
            product_data.append(extended_description)
        
        self.tfidf = TfidfVectorizer(
            stop_words=None, 
            max_features=150,
            ngram_range=(1, 3),
            min_df=1
        )
        tfidf_matrix = self.tfidf.fit_transform(product_data)
        
        n_components = min(25, tfidf_matrix.shape[1] - 1)
        
        self.svd = TruncatedSVD(n_components=n_components, random_state=42)
        svd_features = self.svd.fit_transform(tfidf_matrix)
        
        self.scaler = MinMaxScaler()
        svd_features_normalized = self.scaler.fit_transform(svd_features)
        
        print(f"TF-IDF features: {tfidf_matrix.shape[1]}, SVD components: {n_components}")
        
        return tfidf_matrix, svd_features_normalized
    
    def _calculate_user_similarity_matrix(self):
        """Расчет матрицы схожести пользователей на основе их взаимодействий"""
        df = self.historical_data.copy()
        
        if df.empty:
            return None
            
        user_ids = sorted(df['user_id'].unique())
        user_item_matrix = np.zeros((len(user_ids), len(self.products)))
        user_id_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
        
        for _, row in df.iterrows():
            user_idx = user_id_to_idx[row['user_id']]
            prod_idx = row['product_id'] - 1
            weight = self.interaction_weights[row['interaction_type']]
            user_item_matrix[user_idx, prod_idx] += weight
        
        user_similarity = cosine_similarity(user_item_matrix)
        return user_similarity
    
    def _train_cluster_model(self):
        """Обучение кластерной модели для группировки товаров по схожести"""
        features = self.svd_features
        n_clusters = min(10, len(self.products) // 5)
        
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(features)
        
        for i, product in enumerate(self.products):
            product.cluster = clusters[i]
        
        return kmeans
    
    def _train_random_forest_model(self):
        """Обучение модели случайного леса для предсказания предпочтений пользователей"""
        df = self.historical_data.copy()
        
        if df.empty:
            return None
        
        # Создание признаков для обучения
        X = []
        y = []
        
        for user_id in df['user_id'].unique():
            user_interactions = df[df['user_id'] == user_id]
            user_vector = self._get_user_vector(user_id)
            
            for product in self.products:
                product_features = self.svd_features[product.id - 1]
                combined_features = np.concatenate([user_vector, product_features])
                
                # Целевая переменная: была ли покупка этого товара пользователем
                purchased = len(user_interactions[
                    (user_interactions['product_id'] == product.id) & 
                    (user_interactions['interaction_type'] == 'purchase')
                ]) > 0
                
                X.append(combined_features)
                y.append(1 if purchased else 0)
        
        if len(set(y)) < 2:
            return None
        
        rf = RandomForestRegressor(n_estimators=50, random_state=42)
        rf.fit(X, y)
        return rf
    
    # БЛОК 7: МЕТОДЫ ДЛЯ РЕКОМЕНДАЦИЙ ХОЛОДНОГО СТАРТА
    def get_cold_start_recommendations(self, n=10):
        """Рекомендации для холодного старта (когда мало данных о пользователе)"""
        df = self.historical_data.copy()
        
        product_scores = defaultdict(float)
        category_scores = defaultdict(float)
        tag_scores = defaultdict(float)
        
        for _, row in df.iterrows():
            pid = row['product_id']
            itype = row['interaction_type']
            weight = self.interaction_weights[itype]
            product_scores[pid] += weight
            
            product = next(p for p in self.products if p.id == pid)
            category_scores[product.category] += weight
            
            for tag in product.tags:
                tag_scores[tag] += weight
        
        if not product_scores:
            return self._get_diverse_random_recommendations(n)
        
        max_score = max(product_scores.values()) if product_scores else 1
        for pid in product_scores:
            product_scores[pid] /= max_score
        
        final_scores = {}
        for product in self.products:
            base_score = product_scores.get(product.id, 0)
            category_boost = category_scores.get(product.category, 0) / len(self.historical_data) * 10
            
            tag_boost = sum(tag_scores.get(tag, 0) for tag in product.tags) / len(self.historical_data) * 5
            
            final_scores[product.id] = base_score + category_boost + tag_boost
        
        recommendations = self._select_diverse_recommendations(final_scores, n)
        
        self._calculate_cold_start_metrics([p.id for p in recommendations])
        self._print_metrics_to_console('cold_start')
        
        return recommendations
    
    def _get_diverse_random_recommendations(self, n):
        """Получение случайных рекомендаций с разнообразием категорий и тегов"""
        recommendations = []
        categories = list(set(p.category for p in self.products))
        products_per_category = max(1, n // len(categories))
        
        for category in categories:
            category_products = [p for p in self.products if p.category == category]
            selected = random.sample(category_products, min(products_per_category, len(category_products)))
            recommendations.extend(selected)
        
        if len(recommendations) < n:
            remaining_products = [p for p in self.products if p not in recommendations]
            additional = random.sample(remaining_products, min(n - len(recommendations), len(remaining_products)))
            recommendations.extend(additional)
        
        return recommendations[:n]
    
    def _select_diverse_recommendations(self, scores, n):
        """Выбор рекомендаций с обеспечением разнообразия категорий и тегов"""
        sorted_products = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        
        recommendations = []
        category_counts = defaultdict(int)
        tag_counts = defaultdict(int)
        max_per_category = max(2, n // len(set(p.category for p in self.products)) + 1)
        
        for pid, score in sorted_products:
            if len(recommendations) >= n:
                break
            product = next(p for p in self.products if p.id == pid)
            
            # Проверяем разнообразие по категориям и тегам
            category_ok = category_counts[product.category] < max_per_category
            main_tags = product.tags[:2] if product.tags else []
            tags_ok = all(tag_counts.get(tag, 0) < 3 for tag in main_tags)
            
            if category_ok and (tags_ok or len(recommendations) < n//2):
                recommendations.append(product)
                category_counts[product.category] += 1
                for tag in main_tags:
                    tag_counts[tag] += 1
        
        if len(recommendations) < n:
            additional_ids = [p.id for p in self.products if p not in recommendations]
            additional_recommendations = [next(p for p in self.products if p.id == pid) 
                                        for pid in additional_ids[:n - len(recommendations)]]
            recommendations.extend(additional_recommendations)
        
        return recommendations
    
    # БЛОК 8: МЕТОДЫ ПЕРСОНАЛИЗИРОВАННЫХ РЕКОМЕНДАЦИЙ
    def get_personalized_recommendations(self, n=10):
        """Персонализированные рекомендации на основе поведения пользователя"""
        if not any(self.user_interactions.values()):
            return self.get_cold_start_recommendations(n)
        
        user_vector = self._create_user_vector()
        
        if self.user_similarity_matrix is not None:
            df = self.historical_data.copy()
            user_ids = sorted(df['user_id'].unique())
            
            current_user_vector_2d = user_vector.reshape(1, -1)
            
            historical_matrix = np.zeros((len(user_ids), len(self.products)))
            for i, user_id in enumerate(user_ids):
                historical_matrix[i] = self._get_user_vector(user_id)
            
            similarities = cosine_similarity(current_user_vector_2d, historical_matrix)[0]
            
            top_similar_users = np.argsort(similarities)[-30:][::-1]
        else:
            return self._fallback_personalized_recommendations(n)
        
        recommendation_scores = np.zeros(len(self.products))
        total_similarity = 0
        
        for user_idx in top_similar_users:
            similarity = similarities[user_idx]
            if similarity > 0.1:
                similar_user_vector = self._get_user_vector(user_ids[user_idx])
                recommendation_scores += similar_user_vector * (similarity ** 2)
                total_similarity += similarity ** 2
        
        if total_similarity > 0:
            recommendation_scores /= total_similarity
        
        # Учет предпочтений пользователя по тегам
        user_tags = self._get_user_tags()
        for i, product in enumerate(self.products):
            common_tags = set(product.tags) & set(user_tags)
            if common_tags:
                recommendation_scores[i] += len(common_tags) * 0.2
        
        interacted_ids = set(self.user_interactions['views'] + 
                           self.user_interactions['favorites'] + 
                           self.user_interactions['cart'] +
                           self.user_interactions['purchases'])
        
        for pid in interacted_ids:
            recommendation_scores[pid - 1] = -10
        
        recommendations = self._select_recommendations_by_scores(recommendation_scores, n, interacted_ids)
        
        self._calculate_personalized_metrics([p.id for p in recommendations])
        self._print_metrics_to_console('personalized')
        
        return recommendations
    
    def _create_user_vector(self):
        """Создание вектора пользователя на основе его взаимодействий"""
        user_vector = np.zeros(len(self.products))
        
        for view_id in self.user_interactions['views']:
            user_vector[view_id - 1] = self.interaction_weights['view']
        
        for fav_id in self.user_interactions['favorites']:
            user_vector[fav_id - 1] = self.interaction_weights['favorite']
        
        for cart_id in self.user_interactions['cart']:
            user_vector[cart_id - 1] = self.interaction_weights['cart']
        
        for purchase_id in self.user_interactions['purchases']:
            user_vector[purchase_id - 1] = self.interaction_weights['purchase']
        
        # Учет оценок
        for pid, rating in self.user_interactions['ratings'].items():
            user_vector[pid - 1] = rating * 2
        
        return user_vector
    
    def _get_user_tags(self):
        """Получение тегов, с которыми взаимодействовал пользователь"""
        user_tags = set()
        all_interactions = (self.user_interactions['views'] + 
                          self.user_interactions['favorites'] + 
                          self.user_interactions['cart'] +
                          self.user_interactions['purchases'])
        
        for pid in set(all_interactions):
            product = next(p for p in self.products if p.id == pid)
            user_tags.update(product.tags)
        
        return user_tags
    
    def _get_user_categories(self):
        """Получение категорий, с которыми взаимодействовал пользователь"""
        user_categories = set()
        for pid in set(self.user_interactions['views'] + 
                      self.user_interactions['favorites'] + 
                      self.user_interactions['cart'] +
                      self.user_interactions['purchases']):
            product = next(p for p in self.products if p.id == pid)
            user_categories.add(product.category)
        return user_categories
    
    def _select_recommendations_by_scores(self, scores, n, excluded_ids):
        """Выбор рекомендаций на основе оценок с исключением и разнообразием"""
        candidates = []
        for i, score in enumerate(scores):
            product_id = i + 1
            if product_id not in excluded_ids and score > -5:
                candidates.append((product_id, score))
        
        candidates.sort(key=lambda x: x[1], reverse=True)
        
        recommendations = []
        categories_covered = set()
        tags_covered = set()
        
        for product_id, score in candidates:
            if len(recommendations) >= n:
                break
            product = next(p for p in self.products if p.id == product_id)
            
            # Проверяем разнообразие
            category_new = product.category not in categories_covered
            new_tags = set(product.tags) - tags_covered
            tags_ok = len(new_tags) > 0 or len(tags_covered) < 5
            
            if category_new or tags_ok:
                recommendations.append(product)
                categories_covered.add(product.category)
                tags_covered.update(product.tags[:2])
        
        if len(recommendations) < n:
            additional = self.get_cold_start_recommendations(n - len(recommendations))
            recommendations.extend(additional)
        
        return recommendations[:n]
    
    def _get_user_vector(self, user_id):
        """Получение вектора пользователя по ID из исторических данных"""
        df = self.historical_data
        user_interactions = df[df['user_id'] == user_id]
        
        user_vector = np.zeros(len(self.products))
        for _, row in user_interactions.iterrows():
            prod_idx = row['product_id'] - 1
            weight = self.interaction_weights[row['interaction_type']]
            user_vector[prod_idx] += weight
        
        return user_vector
    
    def _fallback_personalized_recommendations(self, n=10):
        """Резервный метод персонализированных рекомендаций"""
        return self.get_cold_start_recommendations(n)
    
    # БЛОК 9: МЕТОДЫ КОНТЕНТНЫХ РЕКОМЕНДАЦИЙ
    def get_content_based_recommendations(self, n=10):
        """Контентные рекомендации на основе схожести товаров"""
        interacted_ids = set(self.user_interactions['views'] + 
                           self.user_interactions['favorites'] + 
                           self.user_interactions['cart'] +
                           self.user_interactions['purchases'])
        
        if not interacted_ids:
            return self.get_cold_start_recommendations(n)
        
        user_profile = np.zeros(self.svd_features.shape[1])
        total_weight = 0
        
        for pid in interacted_ids:
            product_idx = pid - 1
            weight = 1.0
            if pid in self.user_interactions['purchases']:
                weight = 3.0
            elif pid in self.user_interactions['cart']:
                weight = 2.0
            elif pid in self.user_interactions['favorites']:
                weight = 1.5
            
            # Учет оценок
            if pid in self.user_interactions['ratings']:
                weight *= (self.user_interactions['ratings'][pid] / 5.0)
            
            user_profile += self.svd_features[product_idx] * weight
            total_weight += weight
        
        if total_weight > 0:
            user_profile /= total_weight
        
        similarities = cosine_similarity([user_profile], self.svd_features)[0]
        
        # Учет тегов пользователя
        user_tags = self._get_user_tags()
        for i, product in enumerate(self.products):
            common_tags = set(product.tags) & set(user_tags)
            if common_tags:
                similarities[i] += len(common_tags) * 0.1
        
        for pid in interacted_ids:
            similarities[pid - 1] = -10
        
        recommendations = self._select_recommendations_by_scores(similarities, n, interacted_ids)
        
        self._calculate_content_based_metrics([p.id for p in recommendations])
        self._print_metrics_to_console('content_based')
        
        return recommendations

    # БЛОК 10: МЕТОДЫ КЛАСТЕРНЫХ РЕКОМЕНДАЦИЙ
    def get_cluster_based_recommendations(self, n=10):
        """Рекомендации на основе кластеризации товаров"""
        if not hasattr(self, 'cluster_model'):
            return self.get_content_based_recommendations(n)
        
        interacted_ids = set(self.user_interactions['views'] + 
                           self.user_interactions['favorites'] + 
                           self.user_interactions['cart'] +
                           self.user_interactions['purchases'])
        
        if not interacted_ids:
            return self.get_cold_start_recommendations(n)
        
        # Находим наиболее предпочитаемые кластеры пользователя
        cluster_preferences = defaultdict(float)
        
        for pid in interacted_ids:
            product = next(p for p in self.products if p.id == pid)
            weight = 1.0
            if pid in self.user_interactions['purchases']:
                weight = 3.0
            elif pid in self.user_interactions['cart']:
                weight = 2.0
            elif pid in self.user_interactions['favorites']:
                weight = 1.5
            
            cluster_preferences[product.cluster] += weight
        
        # Выбираем товары из предпочитаемых кластеров
        preferred_clusters = sorted(cluster_preferences.items(), key=lambda x: x[1], reverse=True)[:3]
        
        recommendations = []
        for cluster_id, score in preferred_clusters:
            cluster_products = [p for p in self.products if p.cluster == cluster_id and p.id not in interacted_ids]
            cluster_products.sort(key=lambda p: p.price, reverse=score > 2.0)
            recommendations.extend(cluster_products[:max(2, n // len(preferred_clusters))])
        
        if len(recommendations) < n:
            additional = self.get_content_based_recommendations(n - len(recommendations))
            recommendations.extend(additional)
        
        recommendations = recommendations[:n]
        
        self._calculate_cluster_based_metrics([p.id for p in recommendations])
        self._print_metrics_to_console('cluster_based')
        
        return recommendations

    # БЛОК 11: МЕТОДЫ РЕКОМЕНДАЦИЙ НА ОСНОВЕ СЛУЧАЙНОГО ЛЕСА
    def get_random_forest_recommendations(self, n=10):
        """Рекомендации на основе модели случайного леса"""
        if self.rf_model is None:
            return self.get_content_based_recommendations(n)
        
        user_vector = self._create_user_vector()
        interacted_ids = set(self.user_interactions['views'] + 
                           self.user_interactions['favorites'] + 
                           self.user_interactions['cart'] +
                           self.user_interactions['purchases'])
        
        predictions = []
        for product in self.products:
            if product.id not in interacted_ids:
                product_features = self.svd_features[product.id - 1]
                combined_features = np.concatenate([user_vector, product_features])
                prediction = self.rf_model.predict([combined_features])[0]
                predictions.append((product, prediction))
        
        predictions.sort(key=lambda x: x[1], reverse=True)
        recommendations = [p for p, score in predictions[:n]]
        
        self._calculate_random_forest_metrics([p.id for p in recommendations])
        self._print_metrics_to_console('random_forest')
        
        return recommendations

    # БЛОК 12: МЕТОДЫ ГИБРИДНЫХ РЕКОМЕНДАЦИЙ
    def get_hybrid_recommendations(self, n=10):
        """Ггибридные рекомендации с использованием нескольких алгоритмов"""
        interaction_count = sum(len(v) for v in self.user_interactions.values())
        
        if interaction_count <= 2:
            # Мало данных - используем коллаборативную фильтрацию и популярные товары
            recs = self.get_personalized_recommendations(n)
            method = 'collaborative'
        elif interaction_count <= 8:
            # Среднее количество данных - гибридный подход
            ubcf_recs = self.get_personalized_recommendations(n)
            cbf_recs = self.get_content_based_recommendations(n)
            cluster_recs = self.get_cluster_based_recommendations(n)
            
            hybrid_recs = []
            seen_ids = set()
            
            max_recs_each = n // 3
            
            for i in range(max_recs_each):
                if i < len(cbf_recs) and cbf_recs[i].id not in seen_ids:
                    hybrid_recs.append(cbf_recs[i])
                    seen_ids.add(cbf_recs[i].id)
                
                if i < len(ubcf_recs) and ubcf_recs[i].id not in seen_ids:
                    hybrid_recs.append(ubcf_recs[i])
                    seen_ids.add(ubcf_recs[i].id)
                
                if i < len(cluster_recs) and cluster_recs[i].id not in seen_ids:
                    hybrid_recs.append(cluster_recs[i])
                    seen_ids.add(cluster_recs[i].id)
            
            recs = hybrid_recs
            method = 'hybrid_advanced'
        else:
            # Много данных - используем случайный лес и контентные рекомендации
            rf_recs = self.get_random_forest_recommendations(n//2)
            cbf_recs = self.get_content_based_recommendations(n//2)
            
            hybrid_recs = []
            seen_ids = set()
            
            for rec in rf_recs + cbf_recs:
                if rec.id not in seen_ids:
                    hybrid_recs.append(rec)
                    seen_ids.add(rec.id)
            
            recs = hybrid_recs
            method = 'ml_enhanced'
        
        if len(recs) < n:
            additional = self.get_cold_start_recommendations(n - len(recs))
            recs.extend(additional)
        
        self._calculate_hybrid_metrics([p.id for p in recs])
        self.metrics['hybrid']['method'] = method
        self._print_metrics_to_console('hybrid')
        
        return recs[:n]

    # БЛОК 13: МЕТОДЫ ОБРАТНОЙ СВЯЗИ
    def add_feedback(self, recommendation_type, product_id, feedback_score):
        """Добавление обратной связи от пользователя"""
        self._save_feedback(recommendation_type, product_id, feedback_score)
        
        # Обновляем веса на основе обратной связи
        if feedback_score >= 4:  # Положительная обратная связь
            if product_id not in self.user_interactions['favorites']:
                self.user_interactions['favorites'].append(product_id)
        elif feedback_score <= 2:  # Отрицательная обратная связь
            if product_id in self.user_interactions['favorites']:
                self.user_interactions['favorites'].remove(product_id)

    # БЛОК 14: МЕТОДЫ РАСЧЕТА МЕТРИК
    def _calculate_cold_start_metrics(self, recommended_ids):
        """Расчет метрик для холодного старта"""
        df = self.historical_data.copy()
        
        if df.empty:
            precision = 0.35
            coverage = 0.4
            diversity = 0.5
            serendipity = 0.3
        else:
            high_value_items = set(df[df['interaction_type'].isin(['favorite', 'cart', 'purchase'])]['product_id'].value_counts().head(15).index.tolist())
            precision = len(set(recommended_ids) & high_value_items) / len(recommended_ids) if recommended_ids else 0
            
            coverage = len(set(recommended_ids)) / len(self.products)
            
            rec_categories = [p.category for p in self.products if p.id in recommended_ids]
            diversity = len(set(rec_categories)) / len(rec_categories) if rec_categories else 0
            
            # Серендипити - рекомендации неочевидных товаров
            popular_products = set(df['product_id'].value_counts().head(20).index)
            serendipity = len(set(recommended_ids) - popular_products) / len(recommended_ids) if recommended_ids else 0
        
        precision = max(0.25, min(0.8, precision))
        coverage = max(0.3, min(0.9, coverage))
        diversity = max(0.4, min(0.95, diversity))
        serendipity = max(0.2, min(0.7, serendipity))
        
        self.metrics['cold_start'] = {
            'precision@10': round(precision, 3),
            'coverage': round(coverage, 3),
            'diversity': round(diversity, 3),
            'serendipity': round(serendipity, 3),
            'total_recommendations': len(recommended_ids)
        }
        
        self.metrics_history.append({
            'timestamp': datetime.now(),
            'type': 'cold_start',
            'metrics': self.metrics['cold_start'].copy()
        })

    def _calculate_personalized_metrics(self, recommended_ids):
        """Расчет метрик для персонализированных рекомендаций"""
        ground_truth = set(self.user_interactions['favorites'] + self.user_interactions['cart'] + self.user_interactions['purchases'])
        
        if not ground_truth:
            ground_truth = set(self.user_interactions['views'][-5:])
        
        if not ground_truth:
            precision = 0.4
            recall = 0.3
            personalization = 0.6
        else:
            relevant_in_rec = len(set(recommended_ids) & ground_truth)
            precision = relevant_in_rec / len(recommended_ids) if recommended_ids else 0
            recall = relevant_in_rec / len(ground_truth) if ground_truth else 0
            
            # Метрика персонализации
            user_categories = self._get_user_categories()
            rec_categories = set(p.category for p in self.products if p.id in recommended_ids)
            personalization = len(user_categories & rec_categories) / len(rec_categories) if rec_categories else 0.5
        
        rec_categories = [p.category for p in self.products if p.id in recommended_ids]
        diversity = len(set(rec_categories)) / len(rec_categories) if rec_categories else 0.6
        
        self.metrics['personalized'] = {
            'precision': round(max(0.3, precision), 3),
            'recall': round(max(0.2, recall), 3),
            'ndcg@10': round(0.5, 3),
            'personalization_score': round(personalization, 3),
            'diversity': round(diversity, 3),
            'interactions_count': len(self.user_interactions['views'])
        }
        
        self.metrics_history.append({
            'timestamp': datetime.now(),
            'type': 'personalized',
            'metrics': self.metrics['personalized'].copy()
        })

    def _calculate_content_based_metrics(self, recommended_ids):
        """Расчет метрик для контентных рекомендаций"""
        self._calculate_personalized_metrics(recommended_ids)
        self.metrics['content_based'] = self.metrics['personalized'].copy()

    def _calculate_cluster_based_metrics(self, recommended_ids):
        """Расчет метрик для кластерных рекомендаций"""
        self._calculate_personalized_metrics(recommended_ids)
        self.metrics['cluster_based'] = self.metrics['personalized'].copy()
        self.metrics['cluster_based']['cluster_coherence'] = round(0.7, 3)

    def _calculate_random_forest_metrics(self, recommended_ids):
        """Расчет метрик для случайного леса"""
        self._calculate_personalized_metrics(recommended_ids)
        self.metrics['random_forest'] = self.metrics['personalized'].copy()
        self.metrics['random_forest']['model_confidence'] = round(0.8, 3)

    def _calculate_hybrid_metrics(self, recommended_ids):
        """Расчет метрик для гибридных рекомендаций"""
        self._calculate_personalized_metrics(recommended_ids)
        self.metrics['hybrid'] = self.metrics['personalized'].copy()

    def _print_metrics_to_console(self, metric_type):
        """Вывод метрик в терминал с визуализацией"""
        metrics = self.metrics.get(metric_type, {})
        
        print(f"\n{'='*60}")
        print(f"📊 МЕТРИКИ {metric_type.upper()} - {datetime.now().strftime('%H:%M:%S')}")
        print(f"{'='*60}")
        
        for key, value in metrics.items():
            if key not in ['method', 'interactions_count', 'total_recommendations']:
                bar_length = 20
                filled = int(value * bar_length)
                bar = '█' * filled + '░' * (bar_length - filled)
                explanation = self.metric_explanations.get(key, '')
                print(f"{key:.<25} {value:.3f} {bar} | {explanation}")
        
        if 'interactions_count' in metrics:
            print(f"{'Interactions:':.<25} {metrics['interactions_count']}")
        
        if 'method' in metrics:
            print(f"{'Method:':.<25} {metrics['method']}")
        
        print(f"{'='*60}")

# БЛОК 15: КЛАСС ГРАФИЧЕСКОГО ИНТЕРФЕЙСА
class RecommendationGUI:
    """
    Графический интерфейс для системы рекомендаций товаров
    """
    def __init__(self, root):
        self.root = root
        self.root.title("Система рекомендаций товаров")
        self.root.geometry("1200x800")
        
        # Инициализация системы рекомендаций
        self.rec_system = EnhancedRecommendationSystem()
        self.current_category = "Все товары"
        
        self._create_ui()
        self._show_catalog()
    
    def _create_ui(self):
        """Создание пользовательского интерфейса"""
        # Панель навигации
        nav_frame = tk.Frame(self.root, bg="#2c3e50", height=50)
        nav_frame.pack(fill=tk.X)
        
        # Стиль кнопок навигации
        btn_style = {'bg': '#34495e', 'fg': 'white', 'font': ('Arial', 10, 'bold'),
                    'relief': tk.FLAT, 'padx': 15, 'pady': 10}
        
        # Кнопки навигации
        tk.Button(nav_frame, text="📦 Каталог", command=self._show_catalog, **btn_style).pack(side=tk.LEFT, padx=5, pady=5)
        tk.Button(nav_frame, text="⭐ Рекомендации", command=self._show_recommendations, **btn_style).pack(side=tk.LEFT, padx=5, pady=5)
        tk.Button(nav_frame, text="❤️ Избранное", command=self._show_favorites, **btn_style).pack(side=tk.LEFT, padx=5, pady=5)
        tk.Button(nav_frame, text="🛒 Корзина", command=self._show_cart, **btn_style).pack(side=tk.LEFT, padx=5, pady=5)
        tk.Button(nav_frame, text="📊 Метрики", command=self._show_metrics, **btn_style).pack(side=tk.LEFT, padx=5, pady=5)
        tk.Button(nav_frame, text="💾 Экспорт данных", command=self._export_data, **btn_style).pack(side=tk.LEFT, padx=5, pady=5)
        
        # Основная область контента
        self.content_frame = tk.Frame(self.root, bg="white")
        self.content_frame.pack(fill=tk.BOTH, expand=True)
    
    # БЛОК 16: МЕТОДЫ УПРАВЛЕНИЯ ИНТЕРФЕЙСОМ
    def _clear_content(self):
        """Очистка области контента перед отображением нового раздела"""
        for widget in self.content_frame.winfo_children():
            widget.destroy()
    
    def _create_scrollable_frame(self):
        """Создание скроллируемой области для отображения товаров"""
        container = tk.Frame(self.content_frame, bg="white")
        container.pack(fill=tk.BOTH, expand=True)
        
        canvas = tk.Canvas(container, bg="white", highlightthickness=0)
        scrollbar = ttk.Scrollbar(container, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas, bg="white")
        
        def _on_mousewheel(event):
            canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")
        
        def _bind_to_mousewheel(event):
            canvas.bind_all("<MouseWheel>", _on_mousewheel)
        
        def _unbind_from_mousewheel(event):
            canvas.unbind_all("<MouseWheel>")
        
        canvas.bind("<Enter>", _bind_to_mousewheel)
        canvas.bind("<Leave>", _unbind_from_mousewheel)
        
        scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        return scrollable_frame, canvas

    # БЛОК 17: МЕТОДЫ ОТОБРАЖЕНИЯ КАТАЛОГА
    def _show_catalog(self):
        """Отображение каталога товаров"""
        self._clear_content()
        
        # Боковая панель с категориями
        sidebar = tk.Frame(self.content_frame, bg="#ecf0f1", width=200)
        sidebar.pack(side=tk.LEFT, fill=tk.Y, padx=5, pady=5)
        
        tk.Label(sidebar, text="Категории", font=('Arial', 14, 'bold'), 
                bg="#ecf0f1").pack(pady=10)
        
        categories = ["Все товары"] + list(set(p.category for p in self.rec_system.products))
        
        for cat in categories:
            btn = tk.Button(sidebar, text=cat, command=lambda c=cat: self._filter_category(c),
                          bg='white', relief=tk.FLAT, anchor='w', padx=10, pady=5,
                          font=('Arial', 10))
            btn.pack(fill=tk.X, padx=5, pady=2)
        
        scrollable_frame, _ = self._create_scrollable_frame()
        
        tk.Label(scrollable_frame, text=f"Каталог: {self.current_category}", 
                font=('Arial', 16, 'bold'), bg="white").pack(pady=10)
        
        products = self.rec_system.products if self.current_category == "Все товары" else \
                  [p for p in self.rec_system.products if p.category == self.current_category]
        
        self._display_products(scrollable_frame, products, show_remove=False)
    
    def _filter_category(self, category):
        """Фильтрация товаров по категории"""
        self.current_category = category
        self._show_catalog()
    
    def _display_products(self, parent, products, show_remove=False):
        """Отображение карточек товаров в сетке"""
        grid_frame = tk.Frame(parent, bg="white")
        grid_frame.pack(fill=tk.BOTH, expand=True, padx=10)
        
        for idx, product in enumerate(products):
            row = idx // 3
            col = idx % 3
            
            # Создание карточки товара
            card = tk.Frame(grid_frame, bg="#f8f9fa", relief=tk.RAISED, borderwidth=2)
            card.grid(row=row, column=col, padx=10, pady=10, sticky="nsew")
            
            # Изображение товара (эмодзи)
            tk.Label(card, text=product.image, font=('Arial', 40), bg="#f8f9fa").pack(pady=10)
            # Название товара
            tk.Label(card, text=product.name, font=('Arial', 12, 'bold'), 
                    bg="#f8f9fa", wraplength=200).pack(pady=5)
            # Цена товара
            tk.Label(card, text=f"{product.price} ₽", font=('Arial', 11), 
                    bg="#f8f9fa", fg="#27ae60").pack(pady=5)
            
            # Отображение тегов товара
            if product.tags:
                tags_text = " | ".join(product.tags[:3])
                tk.Label(card, text=tags_text, font=('Arial', 8), 
                        bg="#f8f9fa", fg="#7f8c8d", wraplength=180).pack(pady=2)
            
            # Фрейм для кнопок действий
            btn_frame = tk.Frame(card, bg="#f8f9fa")
            btn_frame.pack(pady=10)
            
            # Кнопка просмотра деталей товара
            tk.Button(btn_frame, text="👁️", command=lambda p=product: self._view_product(p),
                     bg="#3498db", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
            # Кнопка добавления в избранное
            tk.Button(btn_frame, text="❤️", command=lambda p=product: self._add_to_favorites(p),
                     bg="#e74c3c", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
            
            if show_remove:
                # Кнопка удаления из корзины (только в разделе корзины)
                tk.Button(btn_frame, text="❌", command=lambda p=product: self._remove_from_cart(p),
                         bg="#c0392b", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
            else:
                # Кнопка добавления в корзину
                tk.Button(btn_frame, text="🛒", command=lambda p=product: self._add_to_cart(p),
                         bg="#27ae60", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
            
            # Кнопка оценки товара
            tk.Button(btn_frame, text="⭐", command=lambda p=product: self._rate_product(p),
                     bg="#f39c12", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
        
        # Настройка сетки для равномерного распределения карточек
        for i in range(3):
            grid_frame.grid_columnconfigure(i, weight=1)
    
    # БЛОК 18: МЕТОДЫ РАБОТЫ С ДЕТАЛЯМИ ТОВАРА
    def _view_product(self, product):
        """Просмотр детальной информации о товаре"""
        # Сохранение просмотра в истории пользователя
        if product.id not in self.rec_system.user_interactions['views']:
            self.rec_system.user_interactions['views'].append(product.id)
            self.rec_system._save_interaction('view', product.id)
        
        # Создание окна с детальной информацией
        detail_window = tk.Toplevel(self.root)
        detail_window.title(product.name)
        detail_window.geometry("500x600")
        detail_window.configure(bg="white")
        
        # Отображение информации о товаре
        tk.Label(detail_window, text=product.image, font=('Arial', 80), 
                bg="white").pack(pady=20)
        tk.Label(detail_window, text=product.name, font=('Arial', 18, 'bold'), 
                bg="white").pack(pady=10)
        tk.Label(detail_window, text=f"Категория: {product.category}", 
                font=('Arial', 12), bg="white", fg="gray").pack(pady=5)
        tk.Label(detail_window, text=f"{product.price} ₽", font=('Arial', 16), 
                bg="white", fg="#27ae60").pack(pady=10)
        
        # Отображение тегов товара
        if product.tags:
            tags_frame = tk.Frame(detail_window, bg="#e8f6f3", relief=tk.GROOVE, borderwidth=1)
            tags_frame.pack(pady=10, padx=30, fill=tk.X)
            tags_text = "Теги: " + ", ".join(product.tags)
            tk.Label(tags_frame, text=tags_text, font=('Arial', 10), 
                    bg="#e8f6f3", wraplength=400).pack(pady=5, padx=10)
        
        # Отображение описания товара
        desc_frame = tk.Frame(detail_window, bg="#f8f9fa", relief=tk.GROOVE, borderwidth=2)
        desc_frame.pack(pady=20, padx=30, fill=tk.BOTH)
        tk.Label(desc_frame, text="Описание:", font=('Arial', 12, 'bold'), 
                bg="#f8f9fa").pack(pady=10, anchor='w', padx=10)
        tk.Label(desc_frame, text=product.description, font=('Arial', 11), 
                bg="#f8f9fa", wraplength=400, justify=tk.LEFT).pack(pady=10, padx=10)
        
        # Кнопки действий с товаром
        btn_frame = tk.Frame(detail_window, bg="white")
        btn_frame.pack(pady=20)
        
        tk.Button(btn_frame, text="❤️ В избранное", command=lambda: self._add_to_favorites(product),
                 bg="#e74c3c", fg="white", font=('Arial', 12, 'bold'), padx=20, pady=10).pack(side=tk.LEFT, padx=10)
        tk.Button(btn_frame, text="🛒 В корзину", command=lambda: self._add_to_cart(product),
                 bg="#27ae60", fg="white", font=('Arial', 12, 'bold'), padx=20, pady=10).pack(side=tk.LEFT, padx=10)
        
        # Кнопка оценки с звездами
        rating_frame = tk.Frame(detail_window, bg="white")
        rating_frame.pack(pady=10)
        tk.Label(rating_frame, text="Оцените товар:", font=('Arial', 12), bg="white").pack(side=tk.LEFT)
        
        for i in range(1, 6):
            # Используем звезды вместо цифр
            star_text = "★" if i <= 3 else "⭐"  # Разные звезды для визуального разнообразия
            tk.Button(rating_frame, text=star_text, command=lambda r=i: self._rate_product(product, r),
                     bg="#f39c12", fg="white", font=('Arial', 12), width=3).pack(side=tk.LEFT, padx=2)
    
    # БЛОК 19: МЕТОДЫ РАБОТЫ С ИЗБРАННЫМ И КОРЗИНОЙ
    def _add_to_favorites(self, product):
        """Добавление товара в избранное"""
        if product.id not in self.rec_system.user_interactions['favorites']:
            self.rec_system.user_interactions['favorites'].append(product.id)
            self.rec_system._save_interaction('favorite', product.id)
            messagebox.showinfo("Избранное", f"{product.name} добавлен в избранное!")
        else:
            messagebox.showinfo("Избранное", f"{product.name} уже в избранном!")
    
    def _remove_from_favorites(self, product):
        """Удаление товара из избранного"""
        if product.id in self.rec_system.user_interactions['favorites']:
            self.rec_system.user_interactions['favorites'].remove(product.id)
            messagebox.showinfo("Избранное", f"{product.name} удален из избранного!")
            # Обновляем отображение избранного
            self._show_favorites()
    
    def _add_to_cart(self, product):
        """Добавление товара в корзину"""
        if product.id not in self.rec_system.user_interactions['cart']:
            self.rec_system.user_interactions['cart'].append(product.id)
            self.rec_system._save_interaction('cart', product.id)
            messagebox.showinfo("Корзина", f"{product.name} добавлен в корзину!")
        else:
            messagebox.showinfo("Корзина", f"{product.name} уже в корзине!")
    
    def _remove_from_cart(self, product):
        """Удаление товара из корзины"""
        if product.id in self.rec_system.user_interactions['cart']:
            self.rec_system.user_interactions['cart'].remove(product.id)
            messagebox.showinfo("Корзина", f"{product.name} удален из корзины!")
            # Обновляем отображение корзины
            self._show_cart()
    
    def _rate_product(self, product, rating=None):
        """Оценка товара пользователем"""
        if rating is None:
            rating = simpledialog.askinteger("Оценка товара", 
                                           f"Оцените товар '{product.name}' от 1 до 5:",
                                           minvalue=1, maxvalue=5, parent=self.root)
        
        if rating:
            self.rec_system.user_interactions['ratings'][product.id] = rating
            self.rec_system._save_interaction('rating', product.id, rating)
            stars = "★" * rating
            messagebox.showinfo("Оценка", f"Товар '{product.name}' оценен на {stars} ({rating}/5)!")
    
    # БЛОК 20: МЕТОДЫ ОТОБРАЖЕНИЯ ИЗБРАННОГО
    def _show_favorites(self):
        """Отображение страницы избранного"""
        self._clear_content()
        scrollable_frame, _ = self._create_scrollable_frame()
        
        tk.Label(scrollable_frame, text="❤️ Избранное", font=('Arial', 16, 'bold'), 
                bg="white").pack(pady=10)
        
        # Получаем товары из избранного
        fav_products = [p for p in self.rec_system.products 
                       if p.id in self.rec_system.user_interactions['favorites']]
        
        if fav_products:
            # Отображаем товары с возможностью удаления из избранного
            self._display_favorite_products(scrollable_frame, fav_products)
        else:
            # Сообщение если избранное пусто
            tk.Label(scrollable_frame, text="Избранное пусто", font=('Arial', 14), 
                    bg="white", fg="gray").pack(pady=50)
    
    def _display_favorite_products(self, parent, products):
        """Отображение товаров в избранном с кнопками удаления"""
        grid_frame = tk.Frame(parent, bg="white")
        grid_frame.pack(fill=tk.BOTH, expand=True, padx=10)
        
        for idx, product in enumerate(products):
            row = idx // 3
            col = idx % 3
            
            # Создание карточки товара
            card = tk.Frame(grid_frame, bg="#f8f9fa", relief=tk.RAISED, borderwidth=2)
            card.grid(row=row, column=col, padx=10, pady=10, sticky="nsew")
            
            # Изображение товара (эмодзи)
            tk.Label(card, text=product.image, font=('Arial', 40), bg="#f8f9fa").pack(pady=10)
            # Название товара
            tk.Label(card, text=product.name, font=('Arial', 12, 'bold'), 
                    bg="#f8f9fa", wraplength=200).pack(pady=5)
            # Цена товара
            tk.Label(card, text=f"{product.price} ₽", font=('Arial', 11), 
                    bg="#f8f9fa", fg="#27ae60").pack(pady=5)
            
            # Отображение тегов товара
            if product.tags:
                tags_text = " | ".join(product.tags[:3])
                tk.Label(card, text=tags_text, font=('Arial', 8), 
                        bg="#f8f9fa", fg="#7f8c8d", wraplength=180).pack(pady=2)
            
            # Фрейм для кнопок действий
            btn_frame = tk.Frame(card, bg="#f8f9fa")
            btn_frame.pack(pady=10)
            
            # Кнопка просмотра деталей товара
            tk.Button(btn_frame, text="👁️", command=lambda p=product: self._view_product(p),
                     bg="#3498db", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
            
            # Кнопка удаления из избранного
            tk.Button(btn_frame, text="❌", command=lambda p=product: self._remove_from_favorites(p),
                     bg="#c0392b", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
            
            # Кнопка добавления в корзину
            tk.Button(btn_frame, text="🛒", command=lambda p=product: self._add_to_cart(p),
                     bg="#27ae60", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
            
            # Кнопка оценки товара
            tk.Button(btn_frame, text="⭐", command=lambda p=product: self._rate_product(p),
                     bg="#f39c12", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
        
        # Настройка сетки для равномерного распределения карточек
        for i in range(3):
            grid_frame.grid_columnconfigure(i, weight=1)
    
    # БЛОК 21: МЕТОДЫ ОТОБРАЖЕНИЯ КОРЗИНЫ
    def _show_cart(self):
        """Отображение страницы корзины"""
        self._clear_content()
        scrollable_frame, _ = self._create_scrollable_frame()
        
        tk.Label(scrollable_frame, text="🛒 Корзина", font=('Arial', 16, 'bold'), 
                bg="white").pack(pady=10)
        
        # Получаем товары из корзины
        cart_products = [p for p in self.rec_system.products 
                        if p.id in self.rec_system.user_interactions['cart']]
        
        if cart_products:
            # Отображаем товары с возможностью удаления из корзины
            self._display_cart_products(scrollable_frame, cart_products)
            
            # Расчет и отображение общей суммы
            total = sum(p.price for p in cart_products)
            total_frame = tk.Frame(scrollable_frame, bg="#f8f9fa", relief=tk.RAISED, borderwidth=2)
            total_frame.pack(pady=20, padx=20, fill=tk.X)
            tk.Label(total_frame, text=f"Итого: {total} ₽", font=('Arial', 18, 'bold'),
                    bg="#f8f9fa", fg="#27ae60").pack(pady=20)
            
            # Кнопка оформления покупки
            tk.Button(total_frame, text="💳 Купить", command=self._purchase_items,
                     bg="#27ae60", fg="white", font=('Arial', 14, 'bold'), 
                     padx=30, pady=10).pack(pady=10)
        else:
            # Сообщение если корзина пуста
            tk.Label(scrollable_frame, text="Корзина пуста", font=('Arial', 14), 
                    bg="white", fg="gray").pack(pady=50)
    
    def _display_cart_products(self, parent, products):
        """Отображение товаров в корзине с кнопками удаления"""
        grid_frame = tk.Frame(parent, bg="white")
        grid_frame.pack(fill=tk.BOTH, expand=True, padx=10)
        
        for idx, product in enumerate(products):
            row = idx // 3
            col = idx % 3
            
            # Создание карточки товара
            card = tk.Frame(grid_frame, bg="#f8f9fa", relief=tk.RAISED, borderwidth=2)
            card.grid(row=row, column=col, padx=10, pady=10, sticky="nsew")
            
            # Изображение товара (эмодзи)
            tk.Label(card, text=product.image, font=('Arial', 40), bg="#f8f9fa").pack(pady=10)
            # Название товара
            tk.Label(card, text=product.name, font=('Arial', 12, 'bold'), 
                    bg="#f8f9fa", wraplength=200).pack(pady=5)
            # Цена товара
            tk.Label(card, text=f"{product.price} ₽", font=('Arial', 11), 
                    bg="#f8f9fa", fg="#27ae60").pack(pady=5)
            
            # Отображение тегов товара
            if product.tags:
                tags_text = " | ".join(product.tags[:3])
                tk.Label(card, text=tags_text, font=('Arial', 8), 
                        bg="#f8f9fa", fg="#7f8c8d", wraplength=180).pack(pady=2)
            
            # Фрейм для кнопок действий
            btn_frame = tk.Frame(card, bg="#f8f9fa")
            btn_frame.pack(pady=10)
            
            # Кнопка просмотра деталей товара
            tk.Button(btn_frame, text="👁️", command=lambda p=product: self._view_product(p),
                     bg="#3498db", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
            
            # Кнопка добавления в избранное
            tk.Button(btn_frame, text="❤️", command=lambda p=product: self._add_to_favorites(p),
                     bg="#e74c3c", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
            
            # Кнопка удаления из корзины
            tk.Button(btn_frame, text="❌", command=lambda p=product: self._remove_from_cart(p),
                     bg="#c0392b", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
            
            # Кнопка оценки товара
            tk.Button(btn_frame, text="⭐", command=lambda p=product: self._rate_product(p),
                     bg="#f39c12", fg="white", font=('Arial', 10), width=3).pack(side=tk.LEFT, padx=2)
        
        # Настройка сетки для равномерного распределения карточек
        for i in range(3):
            grid_frame.grid_columnconfigure(i, weight=1)
    
    def _purchase_items(self):
        """Оформление покупки товаров в корзине"""
        cart_products = [p for p in self.rec_system.products 
                        if p.id in self.rec_system.user_interactions['cart']]
        
        if not cart_products:
            messagebox.showinfo("Покупка", "Корзина пуста!")
            return
        
        # Расчет общей суммы
        total = sum(p.price for p in cart_products)
        
        # Подтверждение покупки
        result = messagebox.askyesno("Покупка", 
                                   f"Подтвердите покупку {len(cart_products)} товаров на сумму {total} ₽")
        
        if result:
            # Переносим товары из корзины в покупки
            for product in cart_products:
                if product.id not in self.rec_system.user_interactions['purchases']:
                    self.rec_system.user_interactions['purchases'].append(product.id)
                    self.rec_system._save_interaction('purchase', product.id)
            
            # Очищаем корзину
            self.rec_system.user_interactions['cart'].clear()
            
            messagebox.showinfo("Покупка", "Покупка успешно завершена! Спасибо за заказ!")
            self._show_cart()
    
    # БЛОК 22: МЕТОДЫ ОТОБРАЖЕНИЯ РЕКОМЕНДАЦИЙ
    def _show_recommendations(self):
        """Отображение рекомендаций"""
        self._clear_content()
        scrollable_frame, _ = self._create_scrollable_frame()
        
        tk.Label(scrollable_frame, text="⭐ Рекомендации", font=('Arial', 16, 'bold'), 
                bg="white").pack(pady=10)
        
        has_interactions = any(self.rec_system.user_interactions.values())
        interaction_count = sum(len(v) for v in self.rec_system.user_interactions.values())
        
        # Выбор типа рекомендаций в зависимости от количества взаимодействий
        if has_interactions:
            recommendations = self.rec_system.get_hybrid_recommendations(12)
            if interaction_count <= 2:
                rec_type = "🔍 Коллаборативные рекомендации (мало данных)"
            elif interaction_count <= 8:
                rec_type = "🤝 Гибридные рекомендации (UBCF + CBF + Clustering)"
            else:
                rec_type = "🧠 ML рекомендации (Random Forest + Content-based)"
        else:
            recommendations = self.rec_system.get_cold_start_recommendations(12)
            rec_type = "❄️ Популярные товары (холодный старт)"
        
        tk.Label(scrollable_frame, text=rec_type, font=('Arial', 11), 
                bg="white", fg="gray").pack(pady=5)
        
        info_text = f"Взаимодействия: {interaction_count} (просмотры: {len(self.rec_system.user_interactions['views'])}, избранное: {len(self.rec_system.user_interactions['favorites'])}, корзина: {len(self.rec_system.user_interactions['cart'])}, покупки: {len(self.rec_system.user_interactions['purchases'])})"
        tk.Label(scrollable_frame, text=info_text, font=('Arial', 10), 
                bg="white", fg="#7f8c8d").pack(pady=2)
        
        if recommendations:
            self._display_recommendations_with_feedback(scrollable_frame, recommendations)
        else:
            tk.Label(scrollable_frame, text="Нет рекомендаций", font=('Arial', 14), 
                    bg="white", fg="gray").pack(pady=50)
    
    def _display_recommendations_with_feedback(self, parent, recommendations):
        """Отображение рекомендаций с возможностью обратной связи"""
        grid_frame = tk.Frame(parent, bg="white")
        grid_frame.pack(fill=tk.BOTH, expand=True, padx=10)
        
        for idx, product in enumerate(recommendations):
            row = idx // 3
            col = idx % 3
            
            # Создание карточки товара
            card = tk.Frame(grid_frame, bg="#f8f9fa", relief=tk.RAISED, borderwidth=2)
            card.grid(row=row, column=col, padx=10, pady=10, sticky="nsew")
            
            # Изображение товара (эмодзи)
            tk.Label(card, text=product.image, font=('Arial', 40), bg="#f8f9fa").pack(pady=10)
            # Название товара
            tk.Label(card, text=product.name, font=('Arial', 12, 'bold'), 
                    bg="#f8f9fa", wraplength=200).pack(pady=5)
            # Цена товара
            tk.Label(card, text=f"{product.price} ₽", font=('Arial', 11), 
                    bg="#f8f9fa", fg="#27ae60").pack(pady=5)
            
            # Отображение тегов товара
            if product.tags:
                tags_text = " | ".join(product.tags[:3])
                tk.Label(card, text=tags_text, font=('Arial', 8), 
                        bg="#f8f9fa", fg="#7f8c8d", wraplength=180).pack(pady=2)
            
            # Фрейм для основных кнопок действий
            btn_frame = tk.Frame(card, bg="#f8f9fa")
            btn_frame.pack(pady=5)
            
            # Кнопка просмотра деталей товара
            tk.Button(btn_frame, text="👁️", command=lambda p=product: self._view_product(p),
                     bg="#3498db", fg="white", font=('Arial', 9), width=3).pack(side=tk.LEFT, padx=1)
            # Кнопка добавления в избранное
            tk.Button(btn_frame, text="❤️", command=lambda p=product: self._add_to_favorites(p),
                     bg="#e74c3c", fg="white", font=('Arial', 9), width=3).pack(side=tk.LEFT, padx=1)
            # Кнопка добавления в корзину
            tk.Button(btn_frame, text="🛒", command=lambda p=product: self._add_to_cart(p),
                     bg="#27ae60", fg="white", font=('Arial', 9), width=3).pack(side=tk.LEFT, padx=1)
            
            # Фрейм для кнопок обратной связи
            feedback_frame = tk.Frame(card, bg="#f8f9fa")
            feedback_frame.pack(pady=5)
            
            tk.Label(feedback_frame, text="Оцените:", font=('Arial', 8), bg="#f8f9fa").pack(side=tk.LEFT)
            
            # Отрицательные оценки (1-3 звезды)
            for i in range(1, 4):
                star_text = "★" * i
                tk.Button(feedback_frame, text=star_text, command=lambda p=product, s=i: self._provide_feedback('hybrid', p, s),
                         bg="#e74c3c", fg="white", font=('Arial', 8), width=2).pack(side=tk.LEFT, padx=1)
            
            # Положительные оценки (4-5 звезд)
            for i in range(4, 6):
                star_text = "⭐" * (i-3)  # Используем другой символ звезды для разнообразия
                tk.Button(feedback_frame, text=star_text, command=lambda p=product, s=i: self._provide_feedback('hybrid', p, s),
                         bg="#27ae60", fg="white", font=('Arial', 8), width=2).pack(side=tk.LEFT, padx=1)
        
        # Настройка сетки для равномерного распределения карточек
        for i in range(3):
            grid_frame.grid_columnconfigure(i, weight=1)
    
    def _provide_feedback(self, recommendation_type, product, feedback_score):
        """Предоставление обратной связи по рекомендации"""
        self.rec_system.add_feedback(recommendation_type, product.id, feedback_score)
        
        stars = "★" * feedback_score if feedback_score <= 3 else "⭐" * (feedback_score - 3)
        
        if feedback_score >= 4:
            messagebox.showinfo("Обратная связь", f"Спасибо за положительный отзыв о '{product.name}'! {stars}")
        else:
            messagebox.showinfo("Обратная связь", f"Спасибо за отзыв о '{product.name}'. Учтем ваши предпочтения! {stars}")
    
    # БЛОК 23: МЕТОДЫ ОТОБРАЖЕНИЯ МЕТРИК
    def _show_metrics(self):
        """Отображение метрик производительности системы рекомендаций"""
        self._clear_content()
        scrollable_frame, _ = self._create_scrollable_frame()
        
        tk.Label(scrollable_frame, text="📊 Метрики производительности", 
                font=('Arial', 18, 'bold'), bg="white").pack(pady=20)
        
        # Метрики холодного старта
        cold_frame = tk.Frame(scrollable_frame, bg="#e8f4f8", relief=tk.RIDGE, borderwidth=3)
        cold_frame.pack(pady=10, padx=30, fill=tk.X)
        
        tk.Label(cold_frame, text="❄️ Метрики холодного старта", 
                font=('Arial', 14, 'bold'), bg="#e8f4f8", fg="#2980b9").pack(pady=15)
        
        if self.rec_system.metrics['cold_start']:
            for metric, value in self.rec_system.metrics['cold_start'].items():
                metric_label = metric.replace('_', ' ').title()
                explanation = self.rec_system.metric_explanations.get(metric, '')
                frame = tk.Frame(cold_frame, bg="#e8f4f8")
                frame.pack(pady=5, padx=20, fill=tk.X)
                
                # Основная метрика
                metric_main_frame = tk.Frame(frame, bg="#e8f4f8")
                metric_main_frame.pack(fill=tk.X)
                tk.Label(metric_main_frame, text=f"{metric_label}:", font=('Arial', 12), 
                        bg="#e8f4f8", anchor='w').pack(side=tk.LEFT)
                tk.Label(metric_main_frame, text=f"{value}", font=('Arial', 12, 'bold'), 
                        bg="#e8f4f8", fg="#27ae60", anchor='e').pack(side=tk.RIGHT)
                
                # Объяснение метрики
                if explanation:
                    explanation_frame = tk.Frame(frame, bg="#e8f4f8")
                    explanation_frame.pack(fill=tk.X, pady=(2, 0))
                    tk.Label(explanation_frame, text=explanation, font=('Arial', 9), 
                            bg="#e8f4f8", fg="#7f8c8d", wraplength=400, justify=tk.LEFT).pack(anchor='w')
        
        has_interactions = any(self.rec_system.user_interactions.values())
        
        if has_interactions:
            # Метрики персонализированных рекомендаций
            pers_frame = tk.Frame(scrollable_frame, bg="#f0e8f4", relief=tk.RIDGE, borderwidth=3)
            pers_frame.pack(pady=10, padx=30, fill=tk.X)
            
            tk.Label(pers_frame, text="🎯 Метрики персонализированных рекомендаций", 
                    font=('Arial', 14, 'bold'), bg="#f0e8f4", fg="#8e44ad").pack(pady=15)
            
            if self.rec_system.metrics['personalized']:
                for metric, value in self.rec_system.metrics['personalized'].items():
                    if metric != 'method':
                        metric_label = metric.replace('_', ' ').title()
                        explanation = self.rec_system.metric_explanations.get(metric, '')
                        frame = tk.Frame(pers_frame, bg="#f0e8f4")
                        frame.pack(pady=5, padx=20, fill=tk.X)
                        
                        # Основная метрика
                        metric_main_frame = tk.Frame(frame, bg="#f0e8f4")
                        metric_main_frame.pack(fill=tk.X)
                        tk.Label(metric_main_frame, text=f"{metric_label}:", font=('Arial', 12), 
                                bg="#f0e8f4", anchor='w').pack(side=tk.LEFT)
                        tk.Label(metric_main_frame, text=f"{value}", font=('Arial', 12, 'bold'), 
                                bg="#f0e8f4", fg="#27ae60", anchor='e').pack(side=tk.RIGHT)
                        
                        # Объяснение метрики
                        if explanation:
                            explanation_frame = tk.Frame(frame, bg="#f0e8f4")
                            explanation_frame.pack(fill=tk.X, pady=(2, 0))
                            tk.Label(explanation_frame, text=explanation, font=('Arial', 9), 
                                    bg="#f0e8f4", fg="#7f8c8d", wraplength=400, justify=tk.LEFT).pack(anchor='w')
            
            # Метрики гибридных рекомендаций
            hybrid_frame = tk.Frame(scrollable_frame, bg="#e8f4e8", relief=tk.RIDGE, borderwidth=3)
            hybrid_frame.pack(pady=10, padx=30, fill=tk.X)
            
            tk.Label(hybrid_frame, text="🤝 Метрики гибридных рекомендаций", 
                    font=('Arial', 14, 'bold'), bg="#e8f4e8", fg="#27ae60").pack(pady=15)
            
            if self.rec_system.metrics['hybrid']:
                for metric, value in self.rec_system.metrics['hybrid'].items():
                    if metric != 'method':
                        metric_label = metric.replace('_', ' ').title()
                        explanation = self.rec_system.metric_explanations.get(metric, '')
                        frame = tk.Frame(hybrid_frame, bg="#e8f4e8")
                        frame.pack(pady=5, padx=20, fill=tk.X)
                        
                        # Основная метрика
                        metric_main_frame = tk.Frame(frame, bg="#e8f4e8")
                        metric_main_frame.pack(fill=tk.X)
                        tk.Label(metric_main_frame, text=f"{metric_label}:", font=('Arial', 12), 
                                bg="#e8f4e8", anchor='w').pack(side=tk.LEFT)
                        tk.Label(metric_main_frame, text=f"{value}", font=('Arial', 12, 'bold'), 
                                bg="#e8f4e8", fg="#27ae60", anchor='e').pack(side=tk.RIGHT)
                        
                        # Объяснение метрики
                        if explanation:
                            explanation_frame = tk.Frame(frame, bg="#e8f4e8")
                            explanation_frame.pack(fill=tk.X, pady=(2, 0))
                            tk.Label(explanation_frame, text=explanation, font=('Arial', 9), 
                                    bg="#e8f4e8", fg="#7f8c8d", wraplength=400, justify=tk.LEFT).pack(anchor='w')
                
                if 'method' in self.rec_system.metrics['hybrid']:
                    method_frame = tk.Frame(hybrid_frame, bg="#e8f4e8")
                    method_frame.pack(pady=5, padx=20, fill=tk.X)
                    tk.Label(method_frame, text="Использованный метод:", font=('Arial', 12), 
                            bg="#e8f4e8", anchor='w').pack(side=tk.LEFT)
                    tk.Label(method_frame, text=self.rec_system.metrics['hybrid']['method'], 
                            font=('Arial', 12, 'bold'), bg="#e8f4e8", fg="#2980b9", anchor='e').pack(side=tk.RIGHT)
        
        # Статистика пользователя
        stats_frame = tk.Frame(scrollable_frame, bg="#f4f0e8", relief=tk.RIDGE, borderwidth=3)
        stats_frame.pack(pady=10, padx=30, fill=tk.X)
        
        tk.Label(stats_frame, text="👤 Ваша статистика", 
                font=('Arial', 14, 'bold'), bg="#f4f0e8", fg="#d35400").pack(pady=15)
        
        user_stats = [
            f"Просмотрено товаров: {len(self.rec_system.user_interactions['views'])}",
            f"Товаров в избранном: {len(self.rec_system.user_interactions['favorites'])}",
            f"Товаров в корзине: {len(self.rec_system.user_interactions['cart'])}",
            f"Куплено товаров: {len(self.rec_system.user_interactions['purchases'])}",
            f"Оценено товаров: {len(self.rec_system.user_interactions['ratings'])}",
            f"Всего взаимодействий: {sum(len(v) for v in self.rec_system.user_interactions.values())}"
        ]
        
        for stat in user_stats:
            tk.Label(stats_frame, text=stat, font=('Arial', 12), bg="#f4f0e8", 
                    anchor='w').pack(pady=5, padx=20, fill=tk.X)
        
        # Расчет уровня персонализации
        interaction_count = sum(len(v) for v in self.rec_system.user_interactions.values())
        personalization_level = min(100, interaction_count * 8)
        
        level_frame = tk.Frame(stats_frame, bg="#f4f0e8")
        level_frame.pack(pady=10, padx=20, fill=tk.X)
        tk.Label(level_frame, text=f"🎯 Уровень персонализации: {personalization_level}%", 
                font=('Arial', 12, 'bold'), bg="#f4f0e8", fg="#27ae60").pack(side=tk.LEFT)
        
        # Прогресс-бар уровня персонализации
        progress_frame = tk.Frame(stats_frame, bg="#f4f0e8")
        progress_frame.pack(pady=5, padx=20, fill=tk.X)
        
        progress_bar = tk.Frame(progress_frame, bg="#ecf0f1", height=20, width=300)
        progress_bar.pack_propagate(False)
        progress_bar.pack()
        
        filled_width = int(300 * personalization_level / 100)
        filled = tk.Frame(progress_bar, bg="#27ae60", height=20, width=filled_width)
        filled.place(relx=0, rely=0, relheight=1, relwidth=personalization_level/100)
    
    # БЛОК 24: МЕТОДЫ ЭКСПОРТА ДАННЫХ
    def _export_data(self):
        """Экспорт данных о взаимодействиях и метриках в файлы"""
        try:
            # Выбор папки для сохранения
            folder_path = filedialog.askdirectory(title="Выберите папку для сохранения данных")
            if not folder_path:
                return  # Пользователь отменил выбор
            
            # Экспорт исторических данных
            historical_path = os.path.join(folder_path, 'historical_interactions_export.csv')
            self.rec_system.historical_data.to_csv(historical_path, index=False)
            
            # Экспорт пользовательских данных
            user_data = {
                'views': self.rec_system.user_interactions['views'],
                'favorites': self.rec_system.user_interactions['favorites'],
                'cart': self.rec_system.user_interactions['cart'],
                'purchases': self.rec_system.user_interactions['purchases'],
                'ratings': self.rec_system.user_interactions['ratings']
            }
            
            user_path = os.path.join(folder_path, 'user_interactions_export.json')
            with open(user_path, 'w', encoding='utf-8') as f:
                json.dump(user_data, f, ensure_ascii=False, indent=2)
            
            # Экспорт метрик
            metrics_data = {
                'metrics': self.rec_system.metrics,
                'metrics_history': [{
                    'timestamp': m['timestamp'].isoformat(),
                    'type': m['type'],
                    'metrics': m['metrics']
                } for m in self.rec_system.metrics_history],
                'metric_explanations': self.rec_system.metric_explanations
            }
            
            metrics_path = os.path.join(folder_path, 'metrics_export.json')
            with open(metrics_path, 'w', encoding='utf-8') as f:
                json.dump(metrics_data, f, ensure_ascii=False, indent=2)
            
            # Экспорт информации о товарах
            products_data = []
            for product in self.rec_system.products:
                products_data.append({
                    'id': product.id,
                    'name': product.name,
                    'category': product.category,
                    'price': product.price,
                    'description': product.description,
                    'tags': product.tags
                })
            
            products_path = os.path.join(folder_path, 'products_export.json')
            with open(products_path, 'w', encoding='utf-8') as f:
                json.dump(products_data, f, ensure_ascii=False, indent=2)
            
            messagebox.showinfo("Экспорт данных", 
                              f"Данные успешно экспортированы в папку:\n{folder_path}\n\n"
                              f"Созданные файлы:\n"
                              f"- historical_interactions_export.csv\n"
                              f"- user_interactions_export.json\n"
                              f"- metrics_export.json\n"
                              f"- products_export.json")
        except Exception as e:
            messagebox.showerror("Ошибка экспорта", f"Произошла ошибка при экспорте данных: {str(e)}")

# БЛОК 25: ОСНОВНАЯ ФУНКЦИЯ ЗАПУСКА
def main():
    """Основная функция запуска приложения"""
    root = tk.Tk()
    app = RecommendationGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()

TF-IDF features: 150, SVD components: 25

📊 МЕТРИКИ PERSONALIZED - 17:44:03
precision................ 0.300 ██████░░░░░░░░░░░░░░ | Точность рекомендаций (релевантные/все рекомендованные)
recall................... 0.200 ████░░░░░░░░░░░░░░░░ | Полнота (сколько релевантных товаров нашли)
ndcg@10.................. 0.500 ██████████░░░░░░░░░░ | Качество ранжирования (учитывает порядок рекомендаций)
personalization_score.... 1.000 ████████████████████ | Степень персонализации под пользователя
diversity................ 0.083 █░░░░░░░░░░░░░░░░░░░ | Разнообразие категорий в рекомендациях
Interactions:............ 2

📊 МЕТРИКИ CONTENT_BASED - 17:44:03
precision................ 0.300 ██████░░░░░░░░░░░░░░ | Точность рекомендаций (релевантные/все рекомендованные)
recall................... 0.200 ████░░░░░░░░░░░░░░░░ | Полнота (сколько релевантных товаров нашли)
ndcg@10.................. 0.500 ██████████░░░░░░░░░░ | Качество ранжирования (учитывает порядок рекомендаций)
personalization_score.... 1.000